In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 07:34:05 WARN Utils: Your hostname, Davids-MacBook-Pro-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
24/03/03 07:34:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 07:34:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [7]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [8]:
df.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [9]:
df = df.repartition(24)

In [10]:
df.write \
    .mode('overwrite') \
    .parquet('partitioned/')

24/03/02 09:50:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 09:50:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 09:50:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/02 09:50:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 09:50:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 09:50:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 09:50:33 WARN MemoryManager: Total allocation exceeds 95.00%

# 2. Read the partitioned data (Next day)

In [3]:
df = spark.read \
    .parquet('partitioned/')

In [6]:
df.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [7]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_f

In [8]:
df.select('hvfhs_license_num').filter(df['hvfhs_license_num'] == 'HV0003').show()

+-----------------+
|hvfhs_license_num|
+-----------------+
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
|           HV0003|
+-----------------+


In [9]:
from pyspark.sql import functions as F

In [15]:
def crazy_stuff(dispatching_base_num: str):
    """
    This function does some crazy stuff
    :param dispatching_base_num: 
    :return: 
    """
    num = int(dispatching_base_num[1:])
    if num % 2 == 0:
        return 'even'
    else:
        return 'odd'

crazy_stuff_udf = F.udf(crazy_stuff, returnType='string')

In [16]:
df \
    .withColumn('pickup_date', F.to_date('pickup_datetime')) \
    .withColumn('pickup_hour', F.hour('pickup_datetime')) \
    .withColumn('crazy_base_num', crazy_stuff_udf('dispatching_base_num')) \
    .select('pickup_date', 'pickup_hour', 'crazy_base_num') \
    .show(5)

+-----------+-----------+--------------+
|pickup_date|pickup_hour|crazy_base_num|
+-----------+-----------+--------------+
| 2021-01-05|         12|          even|
| 2021-01-02|         14|           odd|
| 2021-01-06|          8|           odd|
| 2021-01-28|          1|           odd|
| 2021-01-17|          9|           odd|
+-----------+-----------+--------------+
